<a href="https://colab.research.google.com/github/sevintanerdi/HFD_project/blob/main/notebooks/HFD_process_out_of_sample_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quantstats


In [ ]:
%cd /content
!rm -rf HFD_project
!git clone https://github.com/sevintanerdi/HFD_project.git

/content
Cloning into 'HFD_project'...
remote: Enumerating objects: 529, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 529 (delta 97), reused 123 (delta 75), pack-reused 369 (from 1)
Receiving objects: 100% (529/529), 28.70 MiB | 23.51 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [ ]:
import os
print(os.getcwd())
print(os.path.exists(input_dir))


/content
True


In [ ]:
import pandas as pd
import numpy as np
import os

# Group 2 quarters
quarters = [
    "2023_Q2",
    "2024_Q1", "2024_Q3",
    "2025_Q3", "2025_Q4"
]

# Paths
input_dir = "HFD_project/data_raw/group2/outofsample_data/"
output_dir = "HFD_project/data_processed/group2_processed_outofsampledata/"
os.makedirs(output_dir, exist_ok=True)

# Process and save files
for q in quarters:
    df = pd.read_parquet(f"{input_dir}data2_{q}.parquet")

    # Set datetime index
    df["datetime"] = pd.to_datetime(df["datetime"])
    df.set_index("datetime", inplace=True)

    # Block trading from 16:50 to 18:10
    pos_flat = np.zeros(len(df))
    mask = (
        (df.index.time >= pd.to_datetime("16:50").time()) &
        (df.index.time <= pd.to_datetime("18:10").time())
    )
    pos_flat[mask] = 1
    df["pos_flat"] = pos_flat

    # Save processed file
    df.to_parquet(f"{output_dir}data2_{q}_processed.parquet")
    print(f"Saved: data2_{q}_processed.parquet")

# Validate processing
print("\nValidating Group 2 break-time rule\n")

for q in quarters:
    df = pd.read_parquet(f"{output_dir}data2_{q}_processed.parquet")
    df.index = pd.to_datetime(df.index)

    break_rows = df.between_time("16:50", "18:10")
    total = len(break_rows)
    correct = (break_rows["pos_flat"] == 1).sum()

    if total == correct:
        print(f"{q}: {correct}/{total} rows correctly masked")
    else:
        print(f"{q}: {correct}/{total} rows masked - ERROR")


Saved: data2_2023_Q2_processed.parquet
Saved: data2_2024_Q1_processed.parquet
Saved: data2_2024_Q3_processed.parquet
Saved: data2_2025_Q3_processed.parquet
Saved: data2_2025_Q4_processed.parquet

Validating Group 2 break-time rule

2023_Q2: 758/758 rows correctly masked
2024_Q1: 1102/1102 rows correctly masked
2024_Q3: 1120/1120 rows correctly masked
2025_Q3: 1121/1121 rows correctly masked
2025_Q4: 377/377 rows correctly masked
